# Objective

1. What are the comorbidities in SCZ patients?
2. Look into EHR for symptom scores? positive versus negative symptoms?

## Notes from Swarali

Looking for schizophrenia comorbidities. Primarily:
- OCD
- Depression
    
## Conclusion (notes from Nish)
1. Identify a cohort of SCZ patients in an observation window
2. Restrict to continuous coverage
3. Look for all claims one year prior to and one year after index date in observation window
4. Look for the following comorbidities
     - OCD
     - Depression
     - Add abuse?
5. Get age for the SCZ cohort and categorise into 5-yr groups
6. Compare age distribution with literature estimates for the US
7. Create weights for patients depending on their age to adjust for their under or over representation in the dataset
8. Create master table - 1 row per patients with their age, calculated weight, and binary (1/0) value for SCZ and each of the comorbidites listed above

## Details

`Observation window:` we are looking for people with a diagnosis of heart failure in the primary or secondary from 1st July 2015 to 31st June 2017

`Index date:` this will be the first occurence of this diagnosis within the observation window

`Look-back:` we will look back 1 year from the index date

`Look-forward:` we will look forward 1 year from the index date

`Continuous coverage:` this is the time period spanning 1 year prior to, and 1 year from the index date, over which patients have to be making medical pharmacy claims




<img src="in/cohort2.png", width=700>

# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [2]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Reference table

Create a reference table to drive the analysis. This will include:
1. Diagnosis codes for SCZ

Convert the table to lowercase

## Dx codes

### Schizophrenia

In [4]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


id     level_1                        level_1_description  level_2  \
0   1  A00 - B999  Certain infectious and parasitic diseases  A00-A09   
1   2  A00 - B999  Certain infectious and parasitic diseases  A00-A09   
2   3  A00 - B999  Certain infectious and parasitic diseases  A00-A09   

              level_2_description   level_3 level_3_description level_4  \
0  Intestinal infectious diseases  A00-A009             Cholera     A00   
1  Intestinal infectious diseases  A00-A009             Cholera    A000   
2  Intestinal infectious diseases  A00-A009             Cholera    A001   

  icd9_mapped_codes                       icd9_description  \
0            No map                                 No map   
1              0010         Cholera due to vibrio cholerae   
2              0011  Cholera due to vibrio cholerae el tor   

                     level_4_short_description_icd10  \
0                                            Cholera   
1  Cholera due to Vibrio cholerae 01 biovar cholerae   
2     Cholera due to Vibrio cholerae 01 biovar eltor   

                      level_4_long_description_icd10   create_ts   update_ts  
0                                            Cholera  2017-11-02  2017-11-02  
1  Cholera due to Vibrio cholerae 01 biovar cholerae  2017-11-02  2017-11-02  
2     Cholera due to Vibrio cholerae 01 biovar eltor  2017-11-02  2017-11-02

In [4]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%schizophrenia%'

Query started at 05:07:17 PM India Standard Time; Query executed in 0.05 m

id    level_1                                level_1_description  \
0  5414  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
1  5415  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
2  5416  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
3  5417  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
4  5418  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
5  5419  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
6  5420  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
7  5422  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
8  5423  F01 - F99  Mental Behavioral and Neurodevelopmental disor...   

   level_2                                level_2_description   level_3  \
0  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
1  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
2  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
3  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
4  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
5  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
6  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
7  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   
8  F20-F29  Schizophrenia schizotypal delusional and other...  F20-F209   

  level_3_description level_4 icd9_mapped_codes  \
0       Schizophrenia     F20            No map   
1       Schizophrenia    F200             29530   
2       Schizophrenia    F201             29510   
3       Schizophrenia    F202             29520   
4       Schizophrenia    F203            No map   
5       Schizophrenia    F205             29560   
6       Schizophrenia    F208            No map   
7       Schizophrenia   F2089             29500   
8       Schizophrenia    F209             29590   

                                    icd9_description  \
0                                             No map   
1            Paranoid type schizophrenia unspecified   
2        Disorganized type schizophrenia unspecified   
3           Catatonic type schizophrenia unspecified   
4                                             No map   
5  Schizophrenic disorders residual type unspecified   
6                                             No map   
7              Simple type schizophrenia unspecified   
8              Unspecified schizophrenia unspecified   

  level_4_short_description_icd10  level_4_long_description_icd10   create_ts  \
0                   Schizophrenia                   Schizophrenia  2017-11-02   
1          Paranoid schizophrenia          Paranoid schizophrenia  2017-11-02   
2      Disorganized schizophrenia      Disorganized schizophrenia  2017-11-02   
3         Catatonic schizophrenia         Catatonic schizophrenia  2017-11-02   
4  Undifferentiated schizophrenia  Undifferentiated schizophrenia  2017-11-02   
5          Residual schizophrenia          Residual schizophrenia  2017-11-02   
6             Other schizophrenia             Other schizophrenia  2017-11-02   
7             Other schizophrenia             Other schizophrenia  2017-11-02   
8       Schizophrenia unspecified       Schizophrenia unspecified  2017-11-02   

    update_ts  
0  2017-11-02  
1  2017-11-02  
2  2017-11-02  
3  2017-11-02  
4  2017-11-02  
5  2017-11-02  
6  2017-11-02  
7  2017-11-02  
8  2017-11-02

In [9]:
%%read_sql

create or replace temporary table st_ssa_ref1 as

select
    'dx' as cat1,
    'scz' cat2,
    null as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_10' as type,
    level_4 as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source
from 
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%schizophrenia%'

Query started at 02:09:10 PM India Standard Time; Query executed in 0.06 m

status
0  Table ST_SSA_REF1 successfully created.

In [10]:
%%read_sql

create or replace temporary table st_ssa_ref2 as

select
    'dx' as cat1,
    'scz' cat2,
    null as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_9' as type,
    icd9_mapped_codes as value,
    icd9_description as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source
from 
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%schizophrenia%'

Query started at 02:09:16 PM India Standard Time; Query executed in 0.05 m

status
0  Table ST_SSA_REF2 successfully created.

### Comorbidities of interest

In [5]:
%%read_sql

select 
    *
from 
    RWD_DB.RWD.ICD_GROUPER
where 
    (level_4_long_description_icd10 ilike '%depressive%'
     and level_4_long_description_icd10 ilike '%disorder%')
    or (level_4_long_description_icd10 ilike '%obsessive%'
        and level_4_long_description_icd10 ilike '%compulsive%'
       and level_4_long_description_icd10 ilike '%disorder%')
    or level_4_long_description_icd10 ilike '%dementia%'
    or (level_4_long_description_icd10 ilike '%alcohol%'
     and level_4_long_description_icd10 ilike '%abuse%')
     or (level_4_long_description_icd10 ilike '%opioid%'
     and level_4_long_description_icd10 ilike '%abuse%')
order by 1

Query started at 02:04:26 PM India Standard Time; Query executed in 0.05 m

id      level_1                                level_1_description  \
0    4828    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
1    4829    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
2    4830    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
3    4831    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
4    4832    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
5    4833    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
6    4834    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
7    4835    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
8    4836    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
9    4837    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
10   4838    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
11   4839    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
12   4848    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
13   4849    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
14   4862    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
15   4863    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
16   4864    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
17   4865    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
18   4866    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
19   4867    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
20   4868    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
21   4869    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
22   4870    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
23   4871    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
24   4872    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
25   4873    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
26   4874    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
27   4875    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
28   4876    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
29   4877    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
..    ...          ...                                                ...   
60   5430    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
61   5480    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
62   5481    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
63   5482    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
64   5483    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
65   5484    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
66   5485    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
67   5486    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
68   5490    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
69   5491    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
70   5492    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
71   5493    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
72   5494    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
73   5495    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
74   5496    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
75   5497    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
76   5498    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
77   5499    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
78   5500    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
79   5501    F01 - F99  Mental Behavioral and Neurodevelopmental disor...   
80 

In [6]:
%%read_sql

--ICD10 temporary table for comorbidities
create or replace temporary table st_ssa_ref3 as

select   
    'dx' as cat1,
    'comorbidity' cat2,
    case
        when (level_4_long_description_icd10 ilike '%depressive%'
              and level_4_long_description_icd10 ilike '%disorder%') then 'depression'
        when (level_4_long_description_icd10 ilike '%obsessive%'
              and level_4_long_description_icd10 ilike '%compulsive%'
              and level_4_long_description_icd10 ilike '%disorder%') then 'OCD'
        when level_4_long_description_icd10 ilike '%dementia%' then 'dementia'
        when (level_4_long_description_icd10 ilike '%alcohol%'
             and level_4_long_description_icd10 ilike '%abuse%') then 'alcohol abuse'
        when (level_4_long_description_icd10 ilike '%opioid%'
             and level_4_long_description_icd10 ilike '%abuse%') then 'opioid abuse'
        end as cat3,
    level_1_description as cat4,
    level_2_description as cat5,
    level_3_description as cat6,
    'icd_10' as type,
    level_4 as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source

from 
    RWD_DB.RWD.ICD_GROUPER
where 
       (level_4_long_description_icd10 ilike '%depressive%'
     and level_4_long_description_icd10 ilike '%disorder%')
    or (level_4_long_description_icd10 ilike '%obsessive%'
        and level_4_long_description_icd10 ilike '%compulsive%'
       and level_4_long_description_icd10 ilike '%disorder%')
    or level_4_long_description_icd10 ilike '%dementia%'
    or (level_4_long_description_icd10 ilike '%alcohol%'
         and level_4_long_description_icd10 ilike '%abuse%')
     or (level_4_long_description_icd10 ilike '%opioid%'
         and level_4_long_description_icd10 ilike '%abuse%')

Query started at 02:07:20 PM India Standard Time; Query executed in 0.08 m

status
0  Table ST_SSA_REF3 successfully created.

In [7]:
%%read_sql

--ICD9 temporary table for comorbidities
create or replace temporary table st_ssa_ref4 as

select   
    'dx' as cat1,
    'comorbidity' cat2,
    case
        when (level_4_long_description_icd10 ilike '%depressive%'
              and level_4_long_description_icd10 ilike '%disorder%') then 'depression'
        when (level_4_long_description_icd10 ilike '%obsessive%'
              and level_4_long_description_icd10 ilike '%compulsive%'
              and level_4_long_description_icd10 ilike '%disorder%') then 'OCD'
        when level_4_long_description_icd10 ilike '%dementia%' then 'dementia'
        when (level_4_long_description_icd10 ilike '%alcohol%'
             and level_4_long_description_icd10 ilike '%abuse%') then 'alcohol abuse'
        when (level_4_long_description_icd10 ilike '%opioid%'
             and level_4_long_description_icd10 ilike '%abuse%') then 'opioid abuse'
        end as cat3,
    level_1_description as cat4,
    level_2_description as cat5,
    level_3_description as cat6,
    'icd_9' as type,
    icd9_mapped_codes as value,
    icd9_description as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source

from 
    RWD_DB.RWD.ICD_GROUPER
where 
    icd9_mapped_codes != 'No map'
    and ((level_4_long_description_icd10 ilike '%depressive%'
         and level_4_long_description_icd10 ilike '%disorder%')
    or (level_4_long_description_icd10 ilike '%obsessive%'
        and level_4_long_description_icd10 ilike '%compulsive%'
        and level_4_long_description_icd10 ilike '%disorder%')
         or level_4_long_description_icd10 ilike '%dementia%'
    or (level_4_long_description_icd10 ilike '%alcohol%'
         and level_4_long_description_icd10 ilike '%abuse%')
     or (level_4_long_description_icd10 ilike '%opioid%'
         and level_4_long_description_icd10 ilike '%abuse%'))

Query started at 02:08:37 PM India Standard Time; Query executed in 0.08 m

status
0  Table ST_SSA_REF4 successfully created.

## Combine

In [11]:
%%read_sql df

select * from st_ssa_ref1
union
select * from st_ssa_ref2
union
select * from st_ssa_ref3
union
select * from st_ssa_ref4

Query started at 02:09:26 PM India Standard Time; Query executed in 0.09 m

cat1         cat2           cat3  \
0     dx          scz           None   
1     dx          scz           None   
2     dx          scz           None   
3     dx          scz           None   
4     dx  comorbidity       dementia   
5     dx  comorbidity       dementia   
6     dx  comorbidity       dementia   
7     dx  comorbidity  alcohol abuse   
8     dx  comorbidity   opioid abuse   
9     dx  comorbidity   opioid abuse   
10    dx  comorbidity       dementia   
11    dx  comorbidity       dementia   
12    dx  comorbidity            OCD   
13    dx  comorbidity       dementia   
14    dx  comorbidity  alcohol abuse   
15    dx  comorbidity   opioid abuse   
16    dx  comorbidity   opioid abuse   
17    dx  comorbidity     depression   
18    dx  comorbidity       dementia   
19    dx  comorbidity  alcohol abuse   
20    dx  comorbidity   opioid abuse   
21    dx  comorbidity   opioid abuse   
22    dx  comorbidity     depression   
23    dx  comorbidity  alcohol abuse   
24    dx  comorbidity  alcohol abuse   
25    dx  comorbidity   opioid abuse   
26    dx  comorbidity     depression   
27    dx  comorbidity       dementia   
28    dx  comorbidity       dementia   
29    dx  comorbidity       dementia   
..   ...          ...            ...   
111   dx  comorbidity       dementia   
112   dx  comorbidity       dementia   
113   dx  comorbidity  alcohol abuse   
114   dx  comorbidity  alcohol abuse   
115   dx  comorbidity       dementia   
116   dx  comorbidity   opioid abuse   
117   dx  comorbidity       dementia   
118   dx  comorbidity     depression   
119   dx  comorbidity  alcohol abuse   
120   dx  comorbidity  alcohol abuse   
121   dx  comorbidity     depression   
122   dx  comorbidity     depression   
123   dx  comorbidity     depression   
124   dx  comorbidity   opioid abuse   
125   dx  comorbidity       dementia   
126   dx  comorbidity     depression   
127   dx  comorbidity       dementia   
128   dx  comorbidity     depression   
129   dx  comorbidity       dementia   
130   dx  comorbidity  alcohol abuse   
131   dx  comorbidity       dementia   
132   dx  comorbidity  alcohol abuse   
133   dx  comorbidity  alcohol abuse   
134   dx  comorbidity   opioid abuse   
135   dx  comorbidity   opioid abuse   
136   dx  comorbidity     depression   
137   dx  comorbidity     depression   
138   dx  comorbidity       dementia   
139   dx  comorbidity            OCD   
140   dx  comorbidity     depression   

                                                  cat4  \
0                                                 None   
1                                                 None   
2                                                 None   
3                                                 None   
4    Mental Behavioral and Neurodevelopmental disor...   
5    Mental Behavioral and Neurodevelopmental disor...   
6    Mental Behavioral and Neurodevelopmental disor...   
7    Mental Behavioral and Neurodevelopmental disor...   
8    Mental Behavioral and Neurodevelopmental disor...   
9    Mental Behavioral and Neurodevelopmental disor...   
10   Mental Behavioral and Neurodevelopmental disor...   
11   Mental Behavioral and Neurodevelopmental disor...   
12   Mental Behavioral and Neurodevelopmental disor...   
13                      Diseases of the nervous system   
14   Mental Behavioral and Neurodevelopmental disor...   
15   Mental Behavioral and Neurodevelopmental disor...   
16   Mental Behavioral and Neurodevelopmental disor...   
17   Mental Behavioral and Neurodevelopmental disor...   
18   Mental Behavioral and Neurodevelopmental disor...   
19   Mental Behavioral and Neurodevelopmental disor...   
20   Mental Behavioral and Neurodevelopmental disor...   
21   Mental Behavioral and Neurodevelopmental disor...   
22   Mental Behavioral and Neurodevelopmental disor...   
23   Mental Behavioral and Neurodevelopmental disor...   
24   Mental Behavioral and Neurodevelopmental disor...   
25   Mental Beh

## Check

In [12]:
# Export to check, and then will inactivate codes that are not relevant and re-upload as final reference table
df.to_excel("out/comorbidities_check.xlsx", index=False)

## Upload into snowflake

In [16]:
scz_ref = pd.read_excel("in/ref.xlsx")

In [17]:
snow.drop_table("st_scz_ref")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_scz_ref;
Table sandbox.st_scz_ref dropped!


In [18]:
snow.upload_dataframe(scz_ref, "st_scz_ref")

Upload successful!


In [19]:
%%read_sql

select *
from st_scz_ref
where deactive_date is not null

Query started at 02:33:53 PM India Standard Time; Query executed in 0.07 m

cat1         cat2           cat3  \
0    dx  comorbidity       dementia   
1    dx  comorbidity       dementia   
2    dx  comorbidity       dementia   
3    dx  comorbidity       dementia   
4    dx  comorbidity       dementia   
5    dx  comorbidity       dementia   
6    dx  comorbidity       dementia   
7    dx  comorbidity       dementia   
8    dx  comorbidity  alcohol abuse   
9    dx  comorbidity       dementia   
10   dx  comorbidity  alcohol abuse   
11   dx  comorbidity     depression   
12   dx  comorbidity  alcohol abuse   
13   dx  comorbidity            OCD   
14   dx  comorbidity  alcohol abuse   
15   dx  comorbidity  alcohol abuse   
16   dx  comorbidity       dementia   
17   dx  comorbidity       dementia   
18   dx  comorbidity       dementia   
19   dx  comorbidity       dementia   
20   dx  comorbidity       dementia   
21   dx  comorbidity       dementia   
22   dx  comorbidity  alcohol abuse   
23   dx  comorbidity   opioid abuse   
24   dx  comorbidity   opioid abuse   
25   dx  comorbidity            OCD   

                                                 cat4  \
0   Mental Behavioral and Neurodevelopmental disor...   
1   Mental Behavioral and Neurodevelopmental disor...   
2                      Diseases of the nervous system   
3   Mental Behavioral and Neurodevelopmental disor...   
4   Mental Behavioral and Neurodevelopmental disor...   
5   Mental Behavioral and Neurodevelopmental disor...   
6   Mental Behavioral and Neurodevelopmental disor...   
7   Mental Behavioral and Neurodevelopmental disor...   
8   Factors influencing health status and contact ...   
9   Mental Behavioral and Neurodevelopmental disor...   
10  Factors influencing health status and contact ...   
11  Mental Behavioral and Neurodevelopmental disor...   
12  Factors influencing health status and contact ...   
13  Mental Behavioral and Neurodevelopmental disor...   
14  Mental Behavioral and Neurodevelopmental disor...   
15  Factors influencing health status and contact ...   
16                     Diseases of the nervous system   
17  Mental Behavioral and Neurodevelopmental disor...   
18  Mental Behavioral and Neurodevelopmental disor...   
19  Mental Behavioral and Neurodevelopmental disor...   
20  Mental Behavioral and Neurodevelopmental disor...   
21                     Diseases of the nervous system   
22  Mental Behavioral and Neurodevelopmental disor...   
23  Mental Behavioral and Neurodevelopmental disor...   
24  Mental Behavioral and Neurodevelopmental disor...   
25  Mental Behavioral and Neurodevelopmental disor...   

                                                 cat5  \
0   Mental and behavioral disorders due to psychoa...   
1   Mental and behavioral disorders due to psychoa...   
2   Other degenerative diseases of the nervous system   
3   Mental and behavioral disorders due to psychoa...   
4   Mental and behavioral disorders due to psychoa...   
5   Mental and behavioral disorders due to psychoa...   
6   Mental and behavioral disorders due to psychoa...   
7   Mental and behavioral disorders due to psychoa...   
8   Persons with potential health hazards related ...   
9   Mental and behavioral disorders due to psychoa...   
10  Persons encountering health services in other ...   
11  Schizophrenia schizotypal delusional and other...   
12  Persons encountering health services in other ...   
13        Disorders of adult personality and behavior   
14  Mental and behavioral disorders due to psychoa...   
15  Persons encountering health services in other ...   
16  Other degenerative diseases of the nervous system   
17  Mental and behavioral disorders due to psychoa...   
18  Mental and behavioral disorders due to psychoa...   
19  Mental and behavioral disorders due to psychoa...   
20  Mental and behavioral disorders due to psychoa...   
21  Other degenerative diseases of the nervous system   
22  Mental and behavioral disorders due to psychoa...   
23  Mental and behavioral disorders due t

# Cohort in obs. window

<img src="in/cohort2.png", width=700>

In [82]:
%%read_sql
create or replace table st_scz_comorb1 as 

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS

    where left (diagnosis,3) in ('F20') 
    and left(patient_id, 5) != 'XXX -'
    limit 1000

Query started at 07:53:26 PM India Standard Time; Query executed in 0.17 m

status
0  Table ST_SCZ_COMORB1 successfully created.

In [80]:
snow.select("select * from RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS where left (diagnosis,3) in ('F20') limit 100")

claim_number                               encrypted_key_1  \
0   con_EP022216736773510  Gz1NV4yRXli2AlrLfDKY9gTHfU1ehN/TRKjSLF2KBdI=   
1   con_EP032018702143506  0TwXRDHGvgE8wvBmuLuuSZoB4gY+e4CZLDOn+V4Jh1k=   
2   con_EP021816728464918  sDbzrfH7wAugsmZpp6ct6GK1aX+i+0RC4ivg34AoJ/A=   
3   con_EI020216271933848  0l+4xmG/aoI/z/9UKV/owMq2jHI+u9UXW2+E9BNrOfY=   
4   con_EI012616247786247  6PwNFHwm3Lv3t6erFkKrKf9rvJuwCl2UiilNg+VSm/Y=   
5   con_EP012816758905938  Xc1JrzMJE89zK/P3LMZ7/n7fgDucwkHkC/phpIlpoX8=   
6   con_EP021316711150167  8r0E3CEsL1zzbBQ6bBp7rxr35b8RRWHvu6GkSCxfgb4=   
7   con_EP020616788665608  HunV5XDehtKXa5MEqrD70vmTuDivdZNLxgJnCjfnQrA=   
8   con_EI012216238015400  67FwMLnsSVtXbJMN1LfwiyPP32PhENU9McZm4LZj8yc=   
9   con_EP041216706519066  oCxpoM3p3lQWUWwRgwJFgIHnXjVokusBPzpLrZt83ow=   
10  con_EI020216272094428  PFakhdH5xJ4EyVftnuS5GLUCvcp/Si7F9L4ceNIMWmk=   
11  con_EP012016728764884  GAa+gkxjx8VB2PvUvnNwfksOFnzPAae9xAsf2DJ2eig=   
12  con_EP020416781192674  EVGkTjaDAM8oaJx+Rx7HtwgAqhqDGV26XS5y4cnxKxc=   
13  con_EI020116269741078  DJOU7SITOj2wnd+o5S2MLTNU5NnpYH0aOQShhnGYuR8=   
14  con_EI012916261243839  yDL+qQLtN8WyJqGi5v7tq0FQy7aubvB7ygzsXe2bU0o=   
15  con_EP040416782017138  sk8E+G/rBZtV3Cc+aIK337WbY/9B0vUwZMiS+cm+/J8=   
16  con_EI020216274259605  EMfVsTz/7rYwOVGir0ReCeOD6sttMfngRITFPNjFo0I=   
17  con_EP020916797526793  rhv2sCoCiBhn12piDXFnI0eSbw5UR/rZhGcmgVlLT2w=   
18         alb_1333525197  qP/op22GvoExS3V/4zbabx9NZr1Lr0fspzAtXNG71TQ=   
19  con_EP022416746250693  ZT7/npU9P63PKBvmWR5MwFzopnyH5SBJMWyjWp4BGiQ=   
20  con_EP060216782601935  mN+4YkpnIkhQRYJdcPHa+Eof6PWe1PkT5vm0I04b04o=   
21  con_EP032216737842752  /q49rYLegs8Aj0hIG6FPBxY0cOtpH1GpMb3BDNOicYc=   
22  con_EI022716257454593  jeEfsMqAXp35C5D2oWpKbYNvd+f8am0Zsc4Fjarc4C0=   
23  con_EP020816792792626  FOLyTb8bCIVUDiKZnx+v3zoVUXVOobvKdZdm8IlCtBA=   
24         alb_1371881044  H2bewDTakzUENsQDkLtgFJOG3frV1lb/5ryy2JgIAmQ=   
25  con_EI012916264175508  29zJV9dw5+eAyCnEmAXulefdgRt6pqS5bgBgm0A4N7o=   
26  con_EP061516723000259  qOGn3bspXbytoT6Q81ZmGva19KGhiZOLALiSKavBjJ0=   
27  con_EP021216709362513  yLj4GaZp7h5noMBlNPalfEUM+LsgmnnMxAO3IpNkks0=   
28  con_EI052316246964864  c5v2m4E713G8WeNrGA0kDkYBhhLxMMNzuwpEB6mSZss=   
29  con_EI012216240052379  SYoOZpSEkDnv93Y+jJmBuf0OxggmUSs1qZJmZRdmCl0=   
..                    ...                                           ...   
70  con_EP022516751681271  N1Fv2NWDz55WI3Z/wcyRKHyu1xcGIdlK/Dq4MJ2neAw=   
71  con_EI020216271950931  sghIKrae1vM93Vi/bqjKvI0NsUwCw0dH5l4aNroYOfc=   
72  con_EP012916763870788  tYOKNXgFyVbgbkdlJ57J969lC8UHgG8dxIU4wmdkhqY=   
73  con_EP081518741146820  inDoz9Vs8WiGeMYl91ui+UUl/C8SVuXv/87OtdEinKQ=   
74  con_EP081418737000411  cXnYeqAFoycfdE/L2SWZ7IdKWeyRmT2vKcjMwachOZw=   
75  con_EP081518741183866  s4JPEi0krOxgUco57hjhgOKozX9FhNpTqUjQsKHQrpI=   
76  con_EI081518241005899  XND8IvnmXa8jAWlBFXD5gH0VhH5xEQ3bIfJYhpkZoP8=   
77  con_EP081518739659744  aNsjiLFNGi/nt6RFVHuhPKBXCT37mfLB5HAUvoUz8B0=   
78  con_EP081518741135661  5J2XaR+9NLryri1i0+5gM44xhA3MutvnkkB3jRbG/aM=   
79  con_EP081518738919846  jau0qOxEZDDnrEq3l7735mDIjjbXCqtGnB/RQxKP0W4=   
80  con_EI081518241063367  lHh27TlayT7KSmNudD+L513UnDbo/XdMU1YVfCPCw6U=   
81  con_EP081418737233266  j6RFgcwmhPq4pVZHRwE1YPHwp+vbqD9t/j/7lnFStYU=   
82  con_EI081518239204792  1cAAQZH8IbzG1EbJVnkMJ0b3gTmDLE6UhWdigTwX4GQ=   
83  con_EP081518738919844  jau0qOxEZDDnrEq3l7735mDIjjbXCqtGnB/RQxKP0W4=   
84  con_EP081518739527145  UdWC55QTjcnQ5QLSNYj5SlaY0xtyprQVr8fAdLZ/Gfg=   
85  con_EP081518741071277  1RWUomh8UPWUozOvj1TRUr6jOhPliaB639f9ZBKrSGg=   
86  con_EP081518740037641  j7gpRoKIN5nVvOg0a+9GUGQDfO1iURXrnKTd6jKS5pY=   
87  con_EP081518739927767  ognPYE/uRr5Uq9tfnBb+7gL8ECQWRbrlMld4xKunLs8=   
88  con_EP081518739660180  hjpekqEElfAuUZx4jO4acdZ8J5UirhE+x8vDh9XZNd4=   
89  con_EP081518739614514  3Xhmulm38NIRHnvR6kzCpLrJsSvGDqdi+8ug/XC/MJQ=   
90  con_EP081518740033153  LabRJwqJuloXlfXONeanDYBQ3/jhDenY7cubVRIXg1c=   
91  con_EP081518738160721  sfE3+Uh5Bod

In [84]:
%%read_sql 
create or replace table st_scz_comorb as
    select a.*,
           b.diagnosis as diag2
from 
    st_scz_comorb1 a
inner join 
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS b
on a.claim_number=b.claim_number

Query started at 10:33:48 PM India Standard Time; Query executed in 4.97 m

status
0  Table ST_SCZ_COMORB successfully created.

In [86]:
%%read_sql df
select * from st_scz_comorb

Query started at 10:41:57 PM India Standard Time; Query executed in 0.46 m

patient_id                              claim_number diagnosis  \
0     BdXTMyPE5MAxNktf  vul_cbfca1bc-ad0d-4274-bc7d-b0888441732e      F201   
1     N9sS+W1B2tEVPxg9                        298309000000210018      F200   
2     802XX1FPi0kxxrAP  vul_9c8453a8-1864-43c4-931d-d107d0190db4      F209   
3     6LDfyOpwc2d9P89v                             vul_178403323      F209   
4     ciEJwAbjMtzxxoqw                     con_EI081518240995725      F200   
5     FyVQzxP1zVnU6sQN                        298309000000116570      F209   
6     15j09uRHyEyeIzK2                     con_EI081418237041543      F200   
7     Qr4S8Z2L9/7HUDtE                     con_EP081418737240811      F200   
8     TVEHky6lDnXwXm0T                     con_EI081518237845026      F200   
9     SHrbB6XRd4XKjiRV                        298310000000299692      F209   
10    wbndLCcV62q59v95                        298309000000312357      F203   
11    15j09uRHyEyeIzK2                     con_EI081418237041543      F200   
12    xF10Ul+xiNqIMshE                     con_EI081418237090431      F200   
13    QcAQTxRuyTqijczO                     con_EP081518741333741      F209   
14    nlFy6cWMKzahNot7                     con_EI081518239834107      F209   
15    pzDkgJKKnFi3pGk6                             vul_178707937     F2089   
16    duLen6bVvbbNBTsL  vul_5b0e4f23-d456-4f0e-902d-2820f76d7ac4      F209   
17    PmL8NEQP5Pvfb5Qp  vul_1cf6820e-ae6a-4cfd-a292-0a85da29d0bc      F209   
18    btHj111s3mFsXe8S                            alb_1748670606      F209   
19    duLen6bVvbbNBTsL  vul_5b0e4f23-d456-4f0e-902d-2820f76d7ac4      F209   
20    +mWmUDR62npLbxWO  vul_2240d307-9602-431a-8255-b76430ee38cc      F200   
21    SfOFg4720YQgEgHK                        298346000000076894      F209   
22    wGMmC9DiByUjT+Rk                     con_EI081618245724532      F209   
23    F2Oamg4aKPZpQlvL                     con_EI081518241490386      F200   
24    jUOZkWOldMc8eAa0                     con_EI081618245624150      F200   
25    m8IVvV/vL76x0HBw                     con_EI081618244306511      F209   
26    KOb/HXpPg6fd6WoS                     con_EI081618244091857      F209   
27    2IlRxM62t1CVhkf8                     con_EI081618244082469     F2089   
28    FKly9HcAlV9ozBby                     con_EP081618742765662      F200   
29    0RUwfJNuSUVjIX4n                        298345000000382896      F209   
...                ...                                       ...       ...   
4068  lzgtD2XLGAYyXV/3                     con_EI081618244017176      F209   
4069  MyCg5oD55xvMWhXQ                        298345000000007377      F200   
4070  0RUwfJNuSUVjIX4n                        298345000000382896      F209   
4071  r1m9BI6g2DgehFYm                        298346000000296477      F209   
4072  QsEUvnjRbCIQ3OKj                     con_EP081618742354117      F200   
4073  +1VTlmLW9nOeEYmk                     con_EP081618744527826      F200   
4074  wLdwQwLphdXvncLK                     con_EP081618744266162      F209   
4075  Ay4k/ttyeQtmkFbN                     con_EP081618744095666      F209   
4076  cYO6GTOU/l6sLsHI                     con_EP081618744405142     F2081   
4077  bTamAl0uBpxLo9We                     con_EP081618745633783      F200   
4078  cIR0hrcgbrAg+JQC                     con_EI081518241556789      F209   
4079  6r8pFQHvw0q7IC+Q                     con_EP081618744179507     F2089   
4080  70n6jMzg0DI2yBcA                     con_EP081618743941944      F200   
4081  PXXf30bjfSTdtC0h                     con_EP081618743929864      F209   
4082  8p6I22LXxtMxMyW6                     con_EI081618245427077      F209   
4083  KOb/HXpPg6fd6WoS                     con_EI081618244091857      F209   
4084  02R+fCw7vWY0hyoI                     con_EI081618243774268      F209   
4085  JCo3dCXBgzuYKYfC                     con_EP081618742353626      F209   
4086  jUOZkWOldMc8eAa0                     con_EI081618245624150      F200   
4087  fvq/m6zsvOoUZ/QF            

In [87]:
df.to_excel("out/other_comorb.xlsx", index=False)

In [18]:
%%read_sql

create or replace table st_scz_obs_cohort as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in (select distinct value from st_scz_ref where cat2 = 'scz' and deactive_date is null)
    and year_of_service between '2015-07-01' and '2017-06-30'

Query started at 04:18:42 PM India Standard Time; Query executed in 21.94 m

status
0  Table ST_SCZ_OBS_COHORT successfully created.

In [19]:
snow.select("select count(distinct patient_id) from st_scz_obs_cohort where left(patient_id, 5) != 'XXX -'")

COUNT(DISTINCT PATIENT_ID)
0                      857660

## Index date

In [20]:
%%read_sql

create or replace table st_scz_index as

select
    patient_id,
    min(year_of_service) as index_date
from
    st_scz_obs_cohort
where
    left(patient_id, 5) != 'XXX -'
group by 1

Query started at 04:42:23 PM India Standard Time; Query executed in 0.19 m

status
0  Table ST_SCZ_INDEX successfully created.

In [21]:
snow.select("select * from st_scz_index order by index_date desc limit 5")

patient_id  index_date
0  DgsKEn71UXdfIib/  2017-06-30
1  NpR7+SoEdOy55nVi  2017-06-30
2  /df25C9Yat3QZeKf  2017-06-30
3  +FPSwMo6r5gWBCPq  2017-06-30
4  jYlDoXdiYVn61Qaa  2017-06-30

In [22]:
snow.select("select count(patient_id) from st_scz_index where left(patient_id, 5) = 'XXX -'")

COUNT(PATIENT_ID)
0                  0

# Continuous coverage

## Cont. cov from medical claims
<img src="in/cohort2.png", width=700>

### Extract submits header

In [37]:
%%read_sql

create or replace table st_scz_cont_med as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    claim_type_code,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) in (select patient_id from st_scz_index)
    and year_of_service between '2014-07-01' and '2018-06-30'

Query started at 05:28:33 PM India Standard Time; Query executed in 14.23 m

status
0  Table ST_SCZ_CONT_MED successfully created.

### Check for duplicates

In [38]:
%%read_sql

select count(*) from st_scz_cont_med

Query started at 05:49:22 PM India Standard Time; Query executed in 0.10 m

COUNT(*)
0  69101373

In [39]:
%%read_sql

with t1 as (select distinct * from st_scz_cont_med)
select count(*) from t1

Query started at 05:49:32 PM India Standard Time; Query executed in 0.33 m

COUNT(*)
0  68949257

In [41]:
%%read_sql

--drop table st_scz_cont_med_unique;
create or replace table st_scz_cont_med_unique as
select distinct * from st_scz_cont_med;
drop table st_scz_cont_med;

Query started at 05:51:11 PM India Standard Time; Query executed in 0.25 mQuery started at 05:51:26 PM India Standard Time; Query executed in 0.05 m

status
0  ST_SCZ_CONT_MED successfully dropped.

### Summarise medical claims

In [42]:
snow.drop_table("st_scz_cont_med_tally")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_scz_cont_med_tally;
Table sandbox.st_scz_cont_med_tally dropped!


In [43]:
%%read_sql

create or replace table st_scz_cont_med_tally as

select
    a.patient_id,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between -365 and -1
           then 1
           else 0
           end) as claim_tally_pre,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between 0 and 365
           then 1
           else 0
           end) as claim_tally_post
from
    st_scz_index a
        join st_scz_cont_med_unique b
            on a.patient_id = b.patient_id
group by 1

Query started at 06:01:38 PM India Standard Time; Query executed in 0.20 m

status
0  Table ST_SCZ_CONT_MED_TALLY successfully created.

### Final counts

In [45]:
a = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally")
b = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally where claim_tally_pre > 0 and claim_tally_post > 0")
c = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally where claim_tally_pre > 1 and claim_tally_post > 1")
d = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally where claim_tally_pre > 2 and claim_tally_post > 2")
e = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally where claim_tally_pre > 3 and claim_tally_post > 3")
f = snow.select("select count(distinct patient_id) from st_scz_cont_med_tally where claim_tally_pre > 4 and claim_tally_post > 4")

lst = ['pts_obs_window',
       '1_claim_pre_and_post',
       '2_claims_pre_and_post',
       '3_claims_pre_and_post',
       '4_claims_pre_and_post',
       '5_claims_pre_and_post',]

pd.concat([a,b,c,d,e,f], keys=lst)

COUNT(DISTINCT PATIENT_ID)
pts_obs_window        0                      857660
1_claim_pre_and_post  0                      715730
2_claims_pre_and_post 0                      646040
3_claims_pre_and_post 0                      586187
4_claims_pre_and_post 0                      538305
5_claims_pre_and_post 0                      494986

## Final cohort

In [46]:
%%read_sql

create or replace table st_scz_cohort as

select
    patient_id,
    index_date
from 
    st_scz_index
where
    patient_id in (select patient_id 
                   from st_scz_cont_med_tally 
                   where claim_tally_pre > 1
                   and claim_tally_pre > 1)

Query started at 06:08:29 PM India Standard Time; Query executed in 0.08 m

status
0  Table ST_SCZ_COHORT successfully created.

In [47]:
snow.select("select count(distinct patient_id) from st_scz_cohort")

COUNT(DISTINCT PATIENT_ID)
0                      662109

# Raw data extracts

Pull all diagnoses data for the cohort of patients from July 1st 2014, to June 31st 2018
<img src="in/cohort2.png", width=700>

## Claim header

In [48]:
%%read_sql

create or replace table st_scz_header_all as 

select
     'XXXXXXXXXXXXXXXX' as patient_id,
     claim_number,
     received_date,
     claim_type_code,
     statement_from,
     statement_to,
     min_service_from,
     max_service_to,
     total_charge,
     total_allowed,
     drg_code,
     type_bill,
     admission_date,
     admit_type_code,
     admit_src_code,
     discharge_hour,
     discharge_status,
     new_medicare_source,
     year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
     claim_number in (select distinct claim_number 
                      from st_scz_cont_med_unique
                      where patient_id in (select patient_id
                                           from st_scz_cohort))

Query started at 06:10:15 PM India Standard Time; Query executed in 6.53 m

status
0  Table ST_SCZ_HEADER_ALL successfully created.

In [49]:
%%read_sql

 begin;
 update st_scz_header_all a
    set a.patient_id = b.patient_id
   from st_scz_cont_med_unique b
  where a.claim_number = b.claim_number;
commit;

Query started at 06:17:17 PM India Standard Time; Query executed in 0.05 mQuery started at 06:17:20 PM India Standard Time; Query executed in 0.71 mQuery started at 06:18:03 PM India Standard Time; Query executed in 0.22 m

status
0  Statement executed successfully.

In [50]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
from st_scz_header_all

Query started at 06:18:31 PM India Standard Time; Query executed in 0.07 m

COUNT(PATIENT_ID)  COUNT(DISTINCT PATIENT_ID)  null_pt_id  faulty_pt_id
0           64772415                      662109           0             0

## Diagnoses

In [51]:
%%read_sql

create or replace table st_scz_dx_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    statement_from,
    statement_to,
    min_service_from,
    max_service_to,
    received_date,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    claim_number in (select distinct claim_number
                     from st_scz_header_all)

Query started at 06:19:10 PM India Standard Time; Query executed in 7.69 m

status
0  Table ST_SCZ_DX_ALL successfully created.

In [52]:
%%read_sql

begin;
update st_scz_dx_all a
set a.patient_id = b.patient_id
from st_scz_header_all b
where a.claim_number = b.claim_number

Query started at 06:27:46 PM India Standard Time; Query executed in 0.05 mQuery started at 06:27:50 PM India Standard Time; Query executed in 0.69 m

number of rows updated  number of multi-joined rows updated
0               203494118                               981136

In [53]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
    
from st_scz_dx_all

Query started at 06:28:51 PM India Standard Time; Query executed in 0.10 m

COUNT(PATIENT_ID)  COUNT(DISTINCT PATIENT_ID)  null_pt_id  faulty_pt_id
0          203494118                      662109           0             0

## Patient raw

In [54]:
%%read_sql

create or replace table st_scz_pat_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    patient_gender,
    patient_dob
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PATIENT
where
    claim_number in (select distinct claim_number
                     from st_scz_header_all)

Query started at 06:29:50 PM India Standard Time; Query executed in 4.83 m

status
0  Table ST_SCZ_PAT_ALL successfully created.

In [55]:
%%read_sql

 begin;
 update st_scz_pat_all a
    set a.patient_id = b.patient_id
   from st_scz_header_all b
  where a.claim_number = b.claim_number;
commit;

Query started at 06:36:40 PM India Standard Time; Query executed in 0.12 mQuery started at 06:36:47 PM India Standard Time; Query executed in 0.81 mQuery started at 06:37:36 PM India Standard Time; Query executed in 0.07 m

status
0  Statement executed successfully.

In [56]:
%%read_sql

create or replace table st_scz_pat_all_unique as

select
    patient_id,
    min(patient_dob) as dob,
    min(patient_gender) as gender
from
    st_scz_pat_all
group by 1

Query started at 06:38:00 PM India Standard Time; Query executed in 0.09 m

status
0  Table ST_SCZ_PAT_ALL_UNIQUE successfully created.

# Master table

## Aggregate comorbidity Dx

In [20]:
snow.select("select distinct cat3 from st_scz_ref")

cat3
0           None
1       dementia
2   opioid abuse
3            OCD
4     depression
5  alcohol abuse

In [22]:
%%read_sql

create or replace table st_scz_comorbidites_agg as 

select
    a.patient_id,
    1 as scz,
    max(case
           when b.diagnosis in (select distinct value
                                from st_scz_ref
                                where cat3 = 'depression' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as depression,
    max(case
           when b.diagnosis in (select distinct value
                                from st_scz_ref
                                where cat3 = 'OCD' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as OCD,
     max(case
           when b.diagnosis in (select distinct value
                                from st_scz_ref
                                where cat3 = 'dementia' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as dementia,
     max(case
           when b.diagnosis in (select distinct value
                                from st_scz_ref
                                where cat3 = 'alcohol abuse' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as alcohol_abuse,
     max(case
           when b.diagnosis in (select distinct value
                                from st_scz_ref
                                where cat3 = 'opioid abuse' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as opioid_abuse
from
    st_scz_cohort a
        join st_scz_dx_all b
            on a.patient_id = b.patient_id
group by 1

Query started at 02:40:54 PM India Standard Time; Query executed in 3.71 m

status
0  Table ST_SCZ_COMORBIDITES_AGG successfully cre...

In [23]:
snow.select("select * from st_scz_comorbidites_agg").to_excel("quick_peek.xlsx", index=False)

## Patient age

In [24]:
%%read_sql

create or replace table st_scz_patient_demographics as 

select
    a.patient_id,
    b.gender,
    datediff(yy, b.dob, a.index_date) as age,
    case
        when datediff(yy, b.dob, a.index_date) between 0 and 4 then '00-04'
        when datediff(yy, b.dob, a.index_date) between 5 and 9 then '05-09'
        when datediff(yy, b.dob, a.index_date) between 10 and 14 then '10-14'
        when datediff(yy, b.dob, a.index_date) between 15 and 19 then '15-19'
        when datediff(yy, b.dob, a.index_date) between 20 and 24 then '20-24'
        when datediff(yy, b.dob, a.index_date) between 25 and 29 then '25-29'
        when datediff(yy, b.dob, a.index_date) between 30 and 34 then '30-34'
        when datediff(yy, b.dob, a.index_date) between 35 and 39 then '35-39'
        when datediff(yy, b.dob, a.index_date) between 40 and 44 then '40-44'
        when datediff(yy, b.dob, a.index_date) between 45 and 49 then '45-49'
        when datediff(yy, b.dob, a.index_date) between 50 and 54 then '50-54'
        when datediff(yy, b.dob, a.index_date) between 55 and 59 then '55-59'
        when datediff(yy, b.dob, a.index_date) between 60 and 64 then '60-64'
        when datediff(yy, b.dob, a.index_date) between 65 and 69 then '65-69'
        when datediff(yy, b.dob, a.index_date) between 70 and 74 then '70-74'
        when datediff(yy, b.dob, a.index_date) between 75 and 79 then '75-79'
        when datediff(yy, b.dob, a.index_date) between 80 and 84 then '80-84'
        when datediff(yy, b.dob, a.index_date) between 85 and 110 then '85+'
        else null
        end as age_cat
from
    st_scz_cohort a
        join st_scz_pat_all_unique b
            on a.patient_id = b.patient_id

Query started at 03:12:57 PM India Standard Time; Query executed in 0.08 m

status
0  Table ST_SCZ_PATIENT_DEMOGRAPHICS successfully...

## Master table

In [25]:
snow.drop_table("st_scz_master_table")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.st_scz_master_table;
Table sandbox.st_scz_master_table dropped!


In [26]:
%%read_sql

create or replace table st_scz_master_table as

select
    a.patient_id,
    null as wt,
    a.gender,
    a.age,
    a.age_cat,
    b.scz,
    b.depression,
    b.OCD,
    b.dementia,
    b.alcohol_abuse,
    b.opioid_abuse
  
from
    st_scz_patient_demographics a
        join st_scz_comorbidites_agg b
            on a.patient_id = b.patient_id

Query started at 03:16:01 PM India Standard Time; Query executed in 0.07 m

status
0  Table ST_SCZ_MASTER_TABLE successfully created.

# Summary results

In [27]:
snow.select("select * from st_scz_master_table limit 3")

patient_id    wt gender  age age_cat  scz  depression  ocd  dementia  \
0  Slpile4aOuDt8d5o  None      M   62   60-64    1           0    0         0   
1  3hyUkt5mPOKfQ585  None      M   55   55-59    1           0    0         0   
2  D4f0BRY6WXUips7T  None      F   58   55-59    1           0    0         0   

   alcohol_abuse  opioid_abuse  
0              0             0  
1              0             0  
2              0             0

In [28]:
%%read_sql

select
    --age_cat,
    sum(scz) as scz_pts,
    sum(depression) as depression,
    round(sum(depression)/sum(scz)*100, 2) as depression_perc,
    sum(OCD) as OCD,
    round(sum(OCD)/sum(scz)*100, 2) as OCD_perc,
     sum(dementia) as dementia,
    round(sum(dementia)/sum(scz)*100, 2) as dementia_perc,
     sum(alcohol_abuse) as alcohol_abuse,
    round(sum(alcohol_abuse)/sum(scz)*100, 2) as alcohol_abuse_perc,
     sum(opioid_abuse) as opioid_abuse,
    round(sum(opioid_abuse)/sum(scz)*100, 2) as opioid_abuse_perc
from
    st_scz_master_table
--group by 1
--order by 1

Query started at 03:20:51 PM India Standard Time; Query executed in 0.05 m

scz_pts  depression  depression_perc    ocd  ocd_perc  dementia  \
0   662109      248678            37.56  12311      1.86     75990   

   dementia_perc  alcohol_abuse  alcohol_abuse_perc  opioid_abuse  \
0          11.48          75480                11.4         20240   

   opioid_abuse_perc  
0               3.06

In [29]:
%%read_sql

select
    age_cat,
    sum(scz) as scz_pts,
    sum(depression) as depression,
    round(sum(depression)/sum(scz)*100, 2) as depression_perc,
    sum(OCD) as OCD,
    round(sum(OCD)/sum(scz)*100, 2) as OCD_perc,
     sum(dementia) as dementia,
    round(sum(dementia)/sum(scz)*100, 2) as dementia_perc,
     sum(alcohol_abuse) as alcohol_abuse,
    round(sum(alcohol_abuse)/sum(scz)*100, 2) as alcohol_abuse_perc,
     sum(opioid_abuse) as opioid_abuse,
    round(sum(opioid_abuse)/sum(scz)*100, 2) as opioid_abuse_perc
from
    st_scz_master_table
group by 1
order by 1

Query started at 03:34:50 PM India Standard Time; Query executed in 0.10 m

age_cat  scz_pts  depression  depression_perc   ocd  ocd_perc  dementia  \
0    00-04       72           6             8.33     0      0.00         4   
1    05-09      603         112            18.57    19      3.15         1   
2    10-14     2509        1068            42.57   119      4.74        12   
3    15-19    11203        5448            48.63   497      4.44        48   
4    20-24    30716       12466            40.58   940      3.06       130   
5    25-29    41732       15966            38.26  1049      2.51       231   
6    30-34    45224       16923            37.42  1064      2.35       306   
7    35-39    47736       17844            37.38  1031      2.16       408   
8    40-44    46632       17557            37.65  1021      2.19       558   
9    45-49    58688       22627            38.55  1088      1.85      1206   
10   50-54    78126       29296            37.50  1226      1.57      2783   
11   55-59    86505       30698            35.49  1210      1.40      5651   
12   60-64    70996       24850            35.00  1140      1.61      8941   
13   65-69    54431       19860            36.49   846      1.55     12305   
14   70-74    35459       13461            37.96   475      1.34     12835   
15   75-79    22459        8818            39.26   295      1.31     11069   
16   80-84    14277        5684            39.81   158      1.11      8754   
17     85+    14739        5993            40.66   133      0.90     10747   
18    None        2           1            50.00     0      0.00         1   

    dementia_perc  alcohol_abuse  alcohol_abuse_perc  opioid_abuse  \
0            5.56              0                0.00             0   
1            0.17              1                0.17             0   
2            0.48             25                1.00             3   
3            0.43            721                6.44           188   
4            0.42           4011               13.06          1271   
5            0.55           6383               15.30          2232   
6            0.68           7184               15.89          2514   
7            0.85           7522               15.76          2426   
8            1.20           6964               14.93          2131   
9            2.05           9036               15.40          2583   
10           3.56          11616               14.87          2750   
11           6.53          10437               12.07          2185   
12          12.59           6215                8.75          1234   
13          22.61           3260                5.99           498   
14          36.20           1295                3.65           149   
15          49.29            522                2.32            45   
16          61.32            205                1.44            17   
17          72.92             83                0.56            14   
18          50.00              0                0.00             0   

    opioid_abuse_perc  
0                0.00  
1                0.00  
2                0.12  
3                1.68  
4                4.14  
5                5.35  
6                5.56  
7                5.08  
8                4.57  
9                4.40  
10               3.52  
11               2.53  
12               1.74  
13               0.91  
14               0.42  
15               0.20  
16               0.12  
17               0.10  
18               0.00

# Notes for Swarali

What you can do from here next week if you wanted to (i'm not even sure if its needed), is to update the wts column in the master table if you wanted to. 

Basically you need the ALS age-distribution in the US (in 5 yr age-groups) i.e. %, and the same for the RWD ALS cohort, and then reweight pts. upwards or downward to shift it to the distribution of the population based study - I haven't worked though the maths but should be easy enough.

And once the weights are updated, you can calculate totals again, but based on sum of weights.